In [3]:
#Task 2.1-User-based/Item-based Collaborative Filtering
#@Saixiong Han

#import train_user_view data
import pandas as pd
import numpy as np

train_user_view_df = pd.read_table('C:/Study/assignment8/train_user_view.txt')
train_user_view_df.head()

,user_id,talk_id,view_time
0,1,10474,07/27/2016 12:27:35
1,1,10488,08/05/2016 12:34:01
2,1,10506,08/07/2016 15:55:57
3,1,10446,08/07/2016 15:58:09
4,1,10506,08/07/2016 16:06:26


In [4]:
new_df = train_user_view_df.sort_values(['user_id','talk_id','view_time'],ascending=[True,True,True])
new_df.head()

,user_id,talk_id,view_time
3,1,10446,08/07/2016 15:58:09
5,1,10446,08/08/2016 18:18:46
6,1,10446,08/09/2016 10:38:38
7,1,10446,08/09/2016 10:39:33
8,1,10446,08/09/2016 16:33:56


In [5]:
#convert the train_user_view.txt into a user-talk matrix.
n_users = train_user_view_df.user_id.unique().shape[0]
n_talks = train_user_view_df.talk_id.unique().shape[0]
print str(n_users) + ' users'
print str(n_talks) + ' talks'
ctr =0
ctr2 = 0
temp = 0
comments = np.zeros((n_users, n_talks))
for row in new_df.itertuples():
    if (row[1]-1)!=temp:
        ctr = 0
        ctr2 = ctr2+1
    comments[ctr2, ctr] = row[2]
    if (ctr+1)<n_talks:
        ctr += 1
    else: 
        ctr = 0
    temp = row[1]-1
comments

37 users
165 talks


array([[ 10446.,  10446.,  10446., ...,      0.,      0.,      0.],
       [ 10636.,  10636.,  10636., ...,  10634.,  10636.,  10636.],
       [ 10516.,  10517.,  10532., ...,      0.,      0.,      0.],
       ..., 
       [ 10634.,      0.,      0., ...,      0.,      0.,      0.],
       [ 10666.,  10666.,  10666., ...,      0.,      0.,      0.],
       [ 10583.,  10591.,  10702., ...,      0.,      0.,      0.]])

In [6]:
sparsity = float(len(comments.nonzero()[0]))
sparsity /= (comments.shape[0] * comments.shape[1])
sparsity *= 100
print 'Sparsity: {:4.2f}%'.format(sparsity)

Sparsity: 20.44%


In [7]:
#split the data into 80/10/10 trainning/validation/test data

def train_rest_split(comments):
    rest = np.zeros(comments.shape)
    train = comments.copy()
    for user in xrange(comments.shape[0]):
        rest_comments = np.random.choice(comments[user, :].nonzero()[0], 
                                        size=20, 
                                        replace=True)
        train[user, rest_comments] = 0.
        rest[user, rest_comments] = comments[user, rest_comments]
        
    # Test and training are truly disjoint
    assert(np.all((train * rest) == 0)) 
    return train, rest

train, rest = train_rest_split(comments)

def valida_test_split(rest):
    test = np.zeros(rest.shape)
    valida = rest.copy()
    for user in xrange(rest.shape[0]):
        test_comments = np.random.choice(rest[user, :].nonzero()[0], 
                                        size=50, 
                                        replace=True)
        valida[user, test_comments] = 0.
        test[user, test_comments] = rest[user, test_comments]
        
    # Test and training are truly disjoint
    assert(np.all((train * rest) == 0)) 
    return valida, test

valida, test = valida_test_split(rest)

In [8]:
#Item-based Collaborative Filtering
#item-similarity matrix will measure the similarity between any two pairs of items.
def slow_similarity(comments, kind='user'):
    if kind == 'user':
        axmax = 0
        axmin = 1
    elif kind == 'talk':
        axmax = 1
        axmin = 0
    sim = np.zeros((comments.shape[axmax], comments.shape[axmax]))
    for u in xrange(comments.shape[axmax]):
        for uprime in xrange(comments.shape[axmax]):
            rui_sqrd = 0.
            ruprimei_sqrd = 0.
            for i in xrange(comments.shape[axmin]):
                sim[u, uprime] = comments[u, i] * comments[uprime, i]
                rui_sqrd += comments[u, i] ** 2
                ruprimei_sqrd += comments[uprime, i] ** 2
            sim[u, uprime] /= rui_sqrd * ruprimei_sqrd
    return sim

def fast_similarity(comments, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = comments.dot(comments.T) + epsilon
    elif kind == 'talk':
        sim = comments.T.dot(comments) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [9]:
%timeit fast_similarity(train, kind='user')

The slowest run took 614.27 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 35.6 µs per loop


In [16]:
%timeit slow_similarity(train, kind='user')

c:\python27\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in double_scalars


1 loop, best of 3: 280 ms per loop


In [10]:
user_similarity_f = fast_similarity(train, kind='user')
talk_similarity_f = fast_similarity(train, kind='talk')
print talk_similarity[:4, :4]

[[ 1.          0.71482388  0.6117559   0.38954011]
 [ 0.71482388  1.          0.66570782  0.63835495]
 [ 0.6117559   0.66570782  1.          0.6383881 ]
 [ 0.38954011  0.63835495  0.6383881   1.        ]]


In [11]:
#user_based collaborating filtering

def predict_slow_simple(comments, similarity, kind='user'):
    pred = np.zeros(comments.shape)
    if kind == 'user':
        for i in xrange(comments.shape[0]):
            for j in xrange(comments.shape[1]):
                pred[i, j] = similarity[i, :].dot(comments[:, j])\
                             /np.sum(np.abs(similarity[i, :]))
        return pred
    elif kind == 'talk':
        for i in xrange(comments.shape[0]):
            for j in xrange(comments.shape[1]):
                pred[i, j] = similarity[j, :].dot(comments[i, :].T)\
                             /np.sum(np.abs(similarity[j, :]))

        return pred

def predict_fast_simple(comments, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(comments) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'talk':
        return comments.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [12]:
%timeit predict_slow_simple(train, user_similarity, kind='user')

10 loops, best of 3: 34.4 ms per loop


In [13]:
%timeit predict_fast_simple(train, user_similarity, kind='user')

The slowest run took 7.90 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 39.6 µs per loop


In [14]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [15]:
#get result from training dataset and validation dataset

talk_prediction = predict_fast_simple(train, talk_similarity, kind='talk')
user_prediction = predict_fast_simple(train, user_similarity, kind='user')

print 'User-based CF MSE: ' + str(get_mse(user_prediction, valida))
print 'Talk-based CF MSE: ' + str(get_mse(talk_prediction, valida))

#get result from training dataset and test dataset

talk_prediction = predict_fast_simple(train, talk_similarity, kind='talk')
user_prediction = predict_fast_simple(train, user_similarity, kind='user')

print 'User-based CF MSE: ' + str(get_mse(user_prediction, test))
print 'Talk-based CF MSE: ' + str(get_mse(talk_prediction, test))

User-based CF MSE: 43719095.5233
Talk-based CF MSE: 37060596.5583
User-based CF MSE: 51428370.1744
Talk-based CF MSE: 71600537.6994


In [14]:
#Task 2.2-Matrix Factorization
#define the matrix_factorization
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T

In [15]:
#R = numpy.array(R)

N = len(train)
M = len(train[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(train, P, Q, K)
nR = numpy.dot(nP, nQ.T)

c:\python27\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: overflow encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: overflow encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: invalid value encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: overflow encountered in double_scalars


In [16]:
print nR

[[        nan         nan         nan ...,         nan         nan
          nan]
 [        nan         nan         nan ...,         nan         nan
          nan]
 [        nan         nan         nan ...,  0.61453516         nan
          nan]
 ..., 
 [        nan         nan         nan ...,  0.70864507         nan
          nan]
 [        nan         nan         nan ...,         nan         nan
          nan]
 [        nan         nan         nan ...,  1.00379919         nan
          nan]]


In [19]:
#Non-Negative Matrix Factorization (NMF)
X = np.array(train)
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
model.fit(X) 
NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [20]:
model.components_

array([[ 38.28690572,  30.10526817,  27.80727828,  30.18528468,
         36.88908141,  20.40348866,  28.28413535,  44.66013347,
         40.34102522,  43.3311295 ,  32.33857706,  31.42895463,
         39.98644538,  45.04979908,  42.75692158,  30.4602856 ,
         29.95700646,  44.73183133,  33.54820991,  43.32133947,
         43.69389077,  43.12866723,  21.84391367,  42.97239072,
         42.16530282,  37.68527506,  41.3566575 ,  44.21206548,
         24.25964018,  47.32851023,  44.97141483,  41.99650738,
         44.0087876 ,  37.18953549,  31.03240261,  30.67583807,
         30.59062712,  43.60782682,  43.21219113,  46.5438701 ,
         35.13924732,  48.11558065,  48.36909776,  32.26192651,
         41.58581946,  34.66575589,  16.78744159,  10.36494072,
         13.53247461,  33.64064657,  48.18910916,  40.53987656,
         35.48618673,  48.21229939,  47.94503189,  46.12637311,
         46.14516418,  41.42495517,  36.58034738,  50.06728276,
         47.97858809,  39.25017129,  16.

In [21]:
model.reconstruction_err_

156500.8684022158